In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')

# Data Preparation 

## Reading datasets

In [ ]:
def read_csv(class_):
    dataFrames = []
    for i in range(1, 8):
        df = pd.read_csv('datasets/' + class_ + '.Cleaned.k' + str(i) + '.csv')
        dataFrames.append(df)
    return dataFrames

### Chiroptera Class: k=1, 2, ...., 7

In [ ]:
chirop_dfs = read_csv('Chiroptera')

### Rodentia Class: k=1, 2, ...., 7

In [ ]:
rodent_dfs = read_csv('Rodentia')

### Aves Class: k=1, 2, ...., 7

In [ ]:
aves_dfs = read_csv('Aves')

# Learning Models

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import f1_score

In [22]:
def get_f1_scores(X, y):
    skf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)
    f1_scores = {'svmrad test': [], 'svmrad train': [], 'svmlin test': [], 'svmlin train': [], 'rf test': [], 'rf train': []}
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
        y_train, y_test = y.iloc[train_index, :], y.iloc[test_index, :]
        svm_rad = SVC(gamma=1/len(X_train.index), kernel='rbf')
        svm_lin = SVC(gamma=1/len(X_train.index), kernel='linear')
        rf = RandomForestClassifier(n_estimators=10)
        svm_rad.fit(X_train, y_train)
        svm_lin.fit(X_train, y_train)
        rf.fit(X_train, y_train)
        y_pred_rad_test = svm_rad.predict(X_test)
        y_pred_rad_train = svm_rad.predict(X_train)
        y_pred_lin_test = svm_lin.predict(X_test)
        y_pred_lin_train = svm_lin.predict(X_train)
        y_pred_rf_test = rf.predict(X_test)
        y_pred_rf_train = rf.predict(X_train)
        f1_scores['svmrad test'].append(f1_score(y_test, y_pred_rad_test, average='micro'))
        f1_scores['svmrad train'].append(f1_score(y_train, y_pred_rad_train, average='micro'))
        f1_scores['svmlin test'].append(f1_score(y_test, y_pred_lin_test, average='micro'))
        f1_scores['svmlin train'].append(f1_score(y_train, y_pred_lin_train, average='micro'))
        f1_scores['rf test'].append(f1_score(y_test, y_pred_rf_test, average='micro'))
        f1_scores['rf train'].append(f1_score(y_train, y_pred_rf_train, average='micro'))
    return f1_scores        

In [ ]:
def main(class_, name):
    clfs_acc = {}
    for i in range(1, len(class_)+1):
        X = class_[i].iloc[:, 3:]
        features = X.columns
        sc = StandardScaler()
        X = sc.fit_transform(X)
        y = pd.DataFrame(.iloc[:, 2])
        X = pd.DataFrame(X, columns=features)
        clfs_acc[name+'.k'+str(i)] = get_f1_scores(X, y)
    return clfs_acc

In [ ]:
import pickle
import dbm

In [ ]:
db = dbm.open('non-hierarchical f1_scores', 'c')
db['Chiroptera'] = pickle.dumps(main(chirop_dfs, 'Chiroptera'))
db['Rodentia'] = pickle.dumps(main(rodent_dfs, 'Rodentia'))
db['Aves'] = pickle.dumps(main(aves_dfs, 'Aves'))
db.close()

## Merged dataset

In [ ]:
def combine_dataset():
    dataFrames = []
    for i in range(1, 8):
        chirop_df = pd.read_csv('datasets/Chiroptera.Cleaned.k' + str(i) + '.csv')
        rodent_df = pd.read_csv('datasets/Rodentia.Cleaned.k' + str(i) + '.csv')
        aves_df = pd.read_csv('datasets/Aves.Cleaned.k' + str(i) + '.csv')
        frames = [chirop_df, rodent_df, aves_df]
        merged_df = pd.concat(frames)
        merged_df.index = range(len(merged_df))
        dataFrames.append(merged_df)
    return dataFrames

In [ ]:
merged_dfs = combine_dataset()
db = dbm.open('merged dataset', 'c')
db['merged datasets'] = pickle.dumps(main(merged_dfs, 'Merged Datasets'))
db.close()